In [ ]:

%pip install transformers pillow -q


In [ ]:
import os
import time
import random
import warnings
warnings.filterwarnings('ignore')

import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import AutoModelForImageClassification, AutoImageProcessor
from transformers import get_cosine_schedule_with_warmup



In [ ]:
# 시드 고정
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



## Config (EDA 기반 최적화)


In [ ]:
CFG = {
    # Paths
    'BASE_DIR': '/home/realtheai/cv_competetion',
    'TRAIN_CSV': '/home/realtheai/cv_competetion/data/train.csv',
    'TEST_CSV': '/home/realtheai/cv_competetion/data/sample_submission.csv',
    'TRAIN_IMG_DIR': '/home/realtheai/cv_competetion/data/train',
    'TEST_IMG_DIR': '/home/realtheai/cv_competetion/data/test',
    
    # Model
    'MODEL_NAME': 'microsoft/dit-base-finetuned-rvlcdip',  # DiT - 문서 특화!
    'NUM_CLASSES': 17,
    
    # Training
    'IMG_SIZE': 224,  # DiT 기본 크기
    'BATCH_SIZE': 8,  # DiT는 LayoutLMv3보다 가벼움
    'EPOCHS': 20,
    'LR': 5e-5,       # Fine-tuning LR
    'WEIGHT_DECAY': 0.01,
    'WARMUP_RATIO': 0.1,
    
    # Device
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("\n" + "="*60)
print("📊 Configuration")
print("="*60)
print(f"모델: {CFG['MODEL_NAME']}")
print(f"IMG_SIZE: {CFG['IMG_SIZE']}")
print(f"BATCH_SIZE: {CFG['BATCH_SIZE']}")
print(f"EPOCHS: {CFG['EPOCHS']}")
print(f"Learning Rate: {CFG['LR']}")
print(f"Device: {CFG['DEVICE']}")
print("="*60)



## Model Setup


In [ ]:
# DiT Image Processor 로드
print("🔄 DiT Image Processor 로드 중...")
processor = AutoImageProcessor.from_pretrained(CFG['MODEL_NAME'])
print("✅ Processor 로드 완료")

# DiT 모델 로드
print("\n🔄 DiT Model 로드 중...")
model = AutoModelForImageClassification.from_pretrained(
    CFG['MODEL_NAME'],
    num_labels=CFG['NUM_CLASSES'],
    ignore_mismatched_sizes=True
).to(CFG['DEVICE'])
print("✅ Model 로드 완료")

print(f"\n📊 모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")
print(f"📊 모델 크기: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")


In [ ]:
class DiTDataset(Dataset):
    def __init__(self, csv_path, img_dir, processor):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.processor = processor
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['ID'])
        image = Image.open(img_path).convert('RGB')
        
        # DiT Processor: 이미지 자동 처리 (정규화, 리사이즈 포함)
        encoding = self.processor(
            images=image,
            return_tensors="pt"
        )
        
        # 배치 차원 제거
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        
        if 'target' in self.df.columns:
            encoding['labels'] = torch.tensor(self.df.iloc[idx]['target'], dtype=torch.long)
        
        return encoding

print("✅ DiTDataset 클래스 정의 완료")


## Data Loading


In [ ]:
# 데이터 로드
train_df = pd.read_csv(CFG['TRAIN_CSV'])
test_df = pd.read_csv(CFG['TEST_CSV'])

# Train/Validation Split
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=SEED,
    stratify=train_df['target']
)

print(f"✅ 데이터 로드 완료")
print(f"   - Train: {len(train_df)}개")
print(f"   - Validation: {len(val_df)}개")
print(f"   - Test: {len(test_df)}개")
print(f"   - 클래스: {CFG['NUM_CLASSES']}개")


## DataLoader Setup


In [ ]:
# Collate 함수 정의
def collate_fn(batch):
    """배치 데이터 처리"""
    batch_dict = {}
    
    for key in batch[0].keys():
        batch_dict[key] = torch.stack([item[key] for item in batch])
    
    return batch_dict

# Dataset 생성
train_dataset = DiTDataset(CFG['TRAIN_CSV'], CFG['TRAIN_IMG_DIR'], processor)
train_dataset.df = train_df.reset_index(drop=True)

val_dataset = DiTDataset(CFG['TRAIN_CSV'], CFG['TRAIN_IMG_DIR'], processor)
val_dataset.df = val_df.reset_index(drop=True)

test_dataset = DiTDataset(CFG['TEST_CSV'], CFG['TEST_IMG_DIR'], processor)

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4
)

print("✅ DataLoader 생성 완료")
print(f"   - Train batches: {len(train_loader)}")
print(f"   - Val batches: {len(val_loader)}")
print(f"   - Test batches: {len(test_loader)}")


## Training Setup


In [ ]:
# Optimizer & Scheduler
optimizer = AdamW(
    model.parameters(),
    lr=CFG['LR'],
    weight_decay=CFG['WEIGHT_DECAY']
)

total_steps = len(train_loader) * CFG['EPOCHS']
warmup_steps = int(total_steps * CFG['WARMUP_RATIO'])

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print("✅ Optimizer & Scheduler 설정 완료")
print(f"   - Total steps: {total_steps}")
print(f"   - Warmup steps: {warmup_steps}")


## Training Functions


In [ ]:
def train_epoch(model, loader, optimizer, scheduler, device):
    """1 Epoch 학습"""
    model.train()
    total_loss = 0
    
    pbar = tqdm(loader, desc="Training")
    for batch in pbar:
        # GPU로 이동
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)


def validate(model, loader, device):
    """Validation"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            batch = {k: v.to(device) for k, v in batch.items()}
            
            outputs = model(**batch)
            loss = outputs.loss
            
            preds = outputs.logits.argmax(dim=-1)
            
            total_loss += loss.item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return avg_loss, accuracy, f1, all_preds, all_labels

print("✅ 학습 함수 정의 완료")


## Training Loop


In [ ]:
# 학습 기록
history = {
    'train_loss': [],
    'val_loss': [],
    'val_acc': [],
    'val_f1': []
}

best_f1 = 0
best_epoch = 0
val_preds = None
val_targets = None

print("\n" + "="*80)
print("🚀 학습 시작!")
print("="*80)

for epoch in range(1, CFG['EPOCHS'] + 1):
    start_time = time.time()
    
    # Train
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, CFG['DEVICE'])
    
    # Validate
    val_loss, val_acc, val_f1, preds, targets = validate(model, val_loader, CFG['DEVICE'])
    
    # 기록
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    # Best 모델 저장
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch
        val_preds = preds
        val_targets = targets
        torch.save(model.state_dict(), f"{CFG['BASE_DIR']}/07_best_model.pth")
        print(f"   ✅ Best 모델 저장! (F1: {best_f1:.4f})")
    
    elapsed_time = time.time() - start_time
    
    print(f"\n[Epoch {epoch}/{CFG['EPOCHS']}] {elapsed_time:.1f}s")
    print(f"   Train Loss: {train_loss:.4f}")
    print(f"   Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")
    print(f"   Best F1: {best_f1:.4f} (Epoch {best_epoch})")
    print("="*80)

print("\n🎉 학습 완료!")
print(f"Best F1: {best_f1:.4f} (Epoch {best_epoch})")


## Results Visualization


In [ ]:
# 학습 곡선
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['val_loss'], label='Val Loss', marker='o')
axes[0].set_title('Loss Curve', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['val_acc'], label='Val Accuracy', marker='o', color='green')
axes[1].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# F1 Score
axes[2].plot(history['val_f1'], label='Val F1', marker='o', color='red')
axes[2].axhline(y=best_f1, color='r', linestyle='--', alpha=0.5, label=f'Best F1: {best_f1:.4f}')
axes[2].set_title('Validation F1 Score', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('F1 Score')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/07_training_curves.png", dpi=150, bbox_inches='tight')
plt.show()

print("✅ 학습 곡선 저장: 07_training_curves.png")


## Wrong Predictions Analysis


In [ ]:
def analyze_wrong_predictions(val_df, preds_list, targets_list, img_dir, meta_df=None, save_path=None):
    """틀린 이미지 시각화 및 패턴 분석"""
    # 틀린 인덱스 찾기
    wrong_indices = [i for i, (pred, target) in enumerate(zip(preds_list, targets_list)) if pred != target]
    
    print(f"\n{'='*60}")
    print(f"📊 검증 결과 분석")
    print(f"{'='*60}")
    print(f"   전체: {len(preds_list)}개")
    print(f"   정답: {len(preds_list) - len(wrong_indices)}개")
    print(f"   오답: {len(wrong_indices)}개 ({len(wrong_indices)/len(preds_list)*100:.1f}%)")
    print(f"   정확도: {(len(preds_list) - len(wrong_indices))/len(preds_list)*100:.1f}%")
    
    if len(wrong_indices) == 0:
        print("\n🎉 모든 예측이 정확합니다!")
        return
    
    # 클래스별 오류 분석
    wrong_by_true_class = {}
    for idx in wrong_indices:
        true_class = targets_list[idx]
        if true_class not in wrong_by_true_class:
            wrong_by_true_class[true_class] = []
        wrong_by_true_class[true_class].append(idx)
    
    print(f"\n📈 클래스별 오답 분석:")
    for class_id in sorted(wrong_by_true_class.keys()):
        count = len(wrong_by_true_class[class_id])
        if meta_df is not None:
            class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
            print(f"   Class {class_id:2d} ({class_name[:20]:20s}): {count:2d}개 오답")
        else:
            print(f"   Class {class_id:2d}: {count:2d}개 오답")
    
    # 틀린 이미지 시각화 (12개)
    if save_path:
        n_show = min(12, len(wrong_indices))
        fig, axes = plt.subplots(3, 4, figsize=(16, 12))
        axes = axes.flatten()
        
        for i, idx in enumerate(wrong_indices[:n_show]):
            img_name = val_df.iloc[idx]['ID']
            img_path = os.path.join(img_dir, img_name)
            
            if os.path.exists(img_path):
                img = Image.open(img_path)
                axes[i].imshow(img)
                
                true_label = f"True: {targets_list[idx]}"
                pred_label = f"Pred: {preds_list[idx]}"
                
                if meta_df is not None:
                    true_class = meta_df[meta_df['target'] == targets_list[idx]]['class_name'].values[0]
                    pred_class = meta_df[meta_df['target'] == preds_list[idx]]['class_name'].values[0]
                    true_label += f"\n({true_class[:15]})"
                    pred_label += f"\n({pred_class[:15]})"
                
                axes[i].set_title(
                    f"{true_label}\n{pred_label}",
                    fontsize=9,
                    color='red',
                    fontweight='bold'
                )
                axes[i].axis('off')
        
        # 빈 subplot 숨기기
        for i in range(n_show, 12):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
        print(f"\n💾 틀린 이미지 저장: {save_path}")

print("✅ analyze_wrong_predictions 함수 정의 완료")

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/training_curves.png", dpi=150, bbox_inches='tight')
plt.show()



In [ ]:
# meta.csv 로드
meta_df = pd.read_csv(f"{CFG['BASE_DIR']}/data/meta.csv")

# Wrong Predictions 분석
analyze_wrong_predictions(
    val_dataset.df,
    val_preds,
    val_targets,
    CFG['TRAIN_IMG_DIR'],
    meta_df=meta_df,
    save_path=f"{CFG['BASE_DIR']}/wrong_predictions.png"
)


## Confusion Matrix


In [ ]:
# Confusion Matrix
cm = confusion_matrix(val_targets, val_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')

# 축 레이블 (클래스 이름)
tick_labels = [f"{i}\n{meta_df[meta_df['target']==i]['class_name'].values[0][:10]}" 
               for i in range(CFG['NUM_CLASSES'])]
plt.xticks(np.arange(CFG['NUM_CLASSES']) + 0.5, tick_labels, rotation=45, ha='right', fontsize=8)
plt.yticks(np.arange(CFG['NUM_CLASSES']) + 0.5, tick_labels, fontsize=8)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/confusion_matrix.png", dpi=150, bbox_inches='tight')
plt.show()



## Inference & Submission


In [ ]:
# Best 모델 로드
model.load_state_dict(torch.load(f"{CFG['BASE_DIR']}/best_model.pth"))
model.eval()

# Test 예측
preds_list = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Inference"):
        # labels 제거 (test에는 없음)
        batch = {k: v.to(CFG['DEVICE']) for k, v in batch.items() if k != 'labels'}
        
        outputs = model(**batch)
        preds = outputs.logits.argmax(dim=-1)
        preds_list.extend(preds.cpu().numpy())

# 제출 파일 생성
submission = pd.read_csv(CFG['TEST_CSV'])
submission['target'] = preds_list
submission.to_csv(f"{CFG['BASE_DIR']}/submission.csv", index=False)
# 예측 분포
print(f"\n📈 Test 클래스별 예측 분포:")
pred_counts = pd.Series(preds_list).value_counts().sort_index()
for class_id, count in pred_counts.items():
    class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
    print(f"   Class {class_id:2d} ({class_name[:25]:25s}): {count:4d}개")
